In [1]:
from utils.generic_utils import *
from utils.basic_modules import *
from utils.UEA_utils import *
from utils.data_plot import *
from SMATE_model import *

import time, os, math
import pydot, pydotplus
import numpy as np
import pandas as pd
import random as rd
import tensorflow as tf
import tensorflow.keras.layers as ll
import tensorflow.keras.backend as K

from decimal import Decimal
from tensorflow.keras.layers import Dense, Conv1D, Conv2D, Conv3D, TimeDistributed
from tensorflow.keras.layers import Input, Dense, Reshape, Flatten, Dropout, multiply, add, GaussianNoise, Lambda
from tensorflow.keras.layers import BatchNormalization, Activation, Embedding, ZeroPadding2D
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras import losses
from tensorflow.keras.losses import mse, binary_crossentropy, categorical_crossentropy
from tensorflow.keras.utils import to_categorical, plot_model, multi_gpu_model

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn import svm, neighbors
from tensorflow.compat.v1 import ConfigProto
from tensorflow.compat.v1 import InteractiveSession
from tensorflow.core.protobuf import rewriter_config_pb2

K.clear_session()
config = tf.ConfigProto(allow_soft_placement=True)
config.gpu_options.per_process_gpu_memory_fraction = 0.9
config.gpu_options.allow_growth = True
off = rewriter_config_pb2.RewriterConfig.OFF
config.graph_options.rewrite_options.arithmetic_optimization = off
sess = tf.Session(config=config)
K.set_session(sess)


In [2]:
sup_ratio = 1 # the supervised ratio in training set, 1 by defaut (fully supervised)
n_epochs = 300

UEA_MTS_List = [
        "ArticularyWordRecognition",
        "AtrialFibrillation",
        "BasicMotions",
        "CharacterTrajectories",
        "Cricket", #5
        #"DuckDuckGeese",
        #"EigenWorms",
        "Epilepsy",
        "EthanolConcentration",
        "ERing",#10
        "FaceDetection",
        "FingerMovements",
        "HandMovementDirection",
        #"Handwriting",
        "Heartbeat",#15
        #"InsectWingbeat",
        #"JapaneseVowels",
        #"Libras",
        "LSST",
        "MotorImagery",#20
        "NATOPS",
        "PenDigits",
        "PEMS-SF",
        "PhonemeSpectra",
        #"RacketSports",#25
        "SelfRegulationSCP1",
        "SelfRegulationSCP2",
        "SpokenArabicDigits",
        "StandWalkJump",        
        #"UWaveGestureLibrary"
]

hyper_paras_map = {
        "ArticularyWordRecognition" : [10, 4], 
        "AtrialFibrillation" : [64, 2],
        "BasicMotions" : [100, 2],
        "CharacterTrajectories" : [30, 2],
        "Cricket" : [100, 4], #5
        #"DuckDuckGeese",
        #"EigenWorms",
        "Epilepsy" : [30, 2],
        "EthanolConcentration" : [50, 2],  
        "ERing": [5, 4],#10
        "FaceDetection" : [8, 16],  
        "FingerMovements" : [5, 8],  
        "HandMovementDirection" : [40, 4],  
        #"Handwriting",
        "Heartbeat" : [5, 16],  #15
        #"InsectWingbeat",
        #"JapaneseVowels",
        #"Libras",
        "LSST" : [4, 4],  
        "MotorImagery" : [100, 8],  #20
        "NATOPS" : [3, 6],
        "PenDigits" : [2, 2],
        "PEMS-SF" : [10, 64],  
        "PhonemeSpectra" : [30, 4],  
        #"RacketSports",#25
        "SelfRegulationSCP1" : [100, 6],  
        "SelfRegulationSCP2" : [100, 6],  
        "SpokenArabicDigits" : [10, 4],  
        "StandWalkJump" : [100, 4],        
        #"UWaveGestureLibrary"
}


---
Input Data format (MTS-others)
---

A single file contains all samples and their labels: ***L * (3 + D)***



- 1st col: sample_id
- 2nd col: timestamps
- 3rd col: label
- after the 4th col: mts vector with D dimensions   


In [3]:
 '''=================================================== Prepare data ========================================================'''
## Prepare UEA data
rep_main = "../Datasets/MTS-UEA/"
ds_name = "PEMS-SF"
rep_ds_train = rep_main + ds_name + "/output_train/"
rep_ds_test = rep_main + ds_name + "/output_test/"
meta_csv = "meta_data.csv"  # the meta data of training/testing set
rep_output = rep_ds_train + "out_results/"  # output results, e.g., training loss, models
#os.system("rm " + rep_output + "running_time.csv")

dataset = get_UEA_dataset(rep_ds_train, rep_ds_test, meta_csv, sup_ratio, mode='load')

class list is ['1.0' '2.0' '3.0' '4.0' '5.0' '6.0' '7.0']
total number of samples is 267
total number of samples is 173


In [4]:
dataset['X_train'].shape

(267, 144, 963)

In [5]:
def running_time(dataset, sample_rate, train_rate, dimension_rate):
    x_train = dataset['X_train']
    x_sup = dataset['X_sup']  # 3-D Array: N * L * D
    x_unsup = dataset['X_unsup']
    y_train = dataset['Y_train']
    y_sup = dataset['Y_sup']
    
    nbr_sample = int(sample_rate * x_train.shape[1])
    nbr_ts_instance = int(train_rate * x_train.shape[0])
    nbr_dimension = int(dimension_rate * x_train.shape[2])
    # vary sample rate
    #x_train = resample_dataset(x_train, nbr_sample)
    #x_sup, x_unsup = x_train, x_train
    #x_test = resample_dataset(x_test, nbr_sample)
    #y_train = y_train[: nbr_ts_instance]

    print("x_train.shape is ", x_train.shape)
    # vary dimension size
    x_train = x_train[:, :, :nbr_dimension]
    x_sup, x_unsup = x_train, x_train
    #x_test = x_test[:, :, :nbr_dimension]
    
    # Bacis Dataset Information and Model Configurations
    train_size = x_train.shape[0] 
    L = x_train.shape[1]
    data_dim = x_train.shape[2]
    n_classes = dataset['n_classes']

    label_size = x_sup.shape[0]
    unlabel_size = x_unsup.shape[0]

    # Two hyper-parameters vary with datasets
    pool_step = hyper_paras_map[ds_name][0]
    d_prime = hyper_paras_map[ds_name][1]
    
    # Build SMATE model
    #K.set_session(sess)
    smate = SMATE(L, data_dim, n_classes, label_size, unlabel_size, 
                  y_sup, sup_ratio, pool_step, d_prime)

    smate.build_model()

    # Train SMATE model
    start = time.time()
    smate.fit(n_epochs, x_train, x_sup, x_unsup)
    print("Training Time for sample_rate (%f2) train_rate (%f2) dimension_size (%f2) is %d" 
          %(sample_rate, train_rate, data_dim, time.time() - start))
    #K.clear_session()
    return time.time() - start
    
def resample_dataset(x, nbr_sample):
    x_sampled = np.zeros(shape=(x.shape[0], nbr_sample, x.shape[2])) # N' * L * D 
    from scipy import signal
    for i in range(x.shape[0]):
        f = signal.resample(x[i], nbr_sample, axis = 0)
        x_sampled[i] = f
    return x_sampled

def save_running_time(ds_name, dataset, save_path, sample_rate, train_rate, dimension_rate):
    df_time = pd.DataFrame(data = np.zeros((1, 5)), columns = ['Dataset', "train_rate", 'sample_rate', 'dimension_rate', 'run_time'])
    run_time = running_time(dataset, sample_rate, train_rate, dimension_rate)
    df_time['Dataset'] = ds_name
    df_time['train_rate'] = train_rate
    df_time['sample_rate'] = sample_rate
    df_time['dimension_rate'] = dimension_rate
    df_time['run_time'] = run_time
    if not os.path.exists(save_path + "SMATE_running_time_full.csv"):
        df_time.to_csv(save_path + "SMATE_running_time_full.csv", index=False)
    else:
        res = pd.read_csv(save_path + "SMATE_running_time_full.csv")
        res = pd.concat((res, df_time))
        res.to_csv(save_path + "SMATE_running_time_full.csv", index=False)
        

In [9]:
# output training time for different sample_rate & train_rate & dimension_rate
# A) vary sample_rate 
train_rate = 1
dimension_rate = 1
for sample_rate in np.linspace(0.1, 1, 10):
    save_running_time(ds_name, dataset, rep_output, sample_rate, train_rate, dimension_rate)
    

W0106 20:01:35.308179 23389298612032 deprecation_wrapper.py:119] From /gpfsdswork/projects/rech/pch/ulz67kb/SMATE_MTS/utils/basic_modules.py:60: The name tf.keras.layers.CuDNNGRU is deprecated. Please use tf.compat.v1.keras.layers.CuDNNGRU instead.

W0106 20:01:35.312204 23389298612032 deprecation.py:506] From /linkhome/rech/genvsq01/ulz67kb/.conda/envs/SMAT_ADE/lib/python3.7/site-packages/tensorflow/python/ops/init_ops.py:1251: calling VarianceScaling.__init__ (from tensorflow.python.ops.init_ops) with dtype is deprecated and will be removed in a future version.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


n_train is 3 ; L = 175 ; D = 3 ; n_class = 4


W0106 20:01:42.028955 23389298612032 training_utils.py:1101] Output dense_8 missing from loss dictionary. We assume this was done on purpose. The fit and evaluate APIs will not be expecting any data to be passed to dense_8.
W0106 20:01:42.063597 23389298612032 deprecation.py:323] From /linkhome/rech/genvsq01/ulz67kb/.conda/envs/SMAT_ADE/lib/python3.7/site-packages/tensorflow/python/ops/math_grad.py:1250: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


n_epochs=3000, batch_size=261, n_sup=261, n_sup=0, steps=3000


/linkhome/rech/genvsq01/ulz67kb/.conda/envs/SMAT_ADE/lib/python3.7/site-packages/tensorflow/python/ops/gradients_util.py:93: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


...

W0106 20:03:58.046490 23389298612032 training_utils.py:1101] Output dense_35 missing from loss dictionary. We assume this was done on purpose. The fit and evaluate APIs will not be expecting any data to be passed to dense_35.


n_epochs=3000, batch_size=261, n_sup=261, n_sup=0, steps=3000


/linkhome/rech/genvsq01/ulz67kb/.conda/envs/SMAT_ADE/lib/python3.7/site-packages/tensorflow/python/ops/gradients_util.py:93: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Training Time for sample_rate (0.4000002) train_rate (1.0000002)  is 6
n_train is 3 ; L = 875 ; D = 3 ; n_class = 4


KeyboardInterrupt: 

In [5]:
# B) vary train_rate
sample_rate = 1
dimension_rate = 1
for train_rate in np.linspace(0.1, 1, 10):
    save_running_time(ds_name, dataset, rep_output, sample_rate, train_rate, dimension_rate)

W0105 22:36:37.122359 22666830645056 deprecation_wrapper.py:119] From /gpfsdswork/projects/rech/pch/ulz67kb/SMATE_MTS/utils/basic_modules.py:60: The name tf.keras.layers.CuDNNGRU is deprecated. Please use tf.compat.v1.keras.layers.CuDNNGRU instead.

W0105 22:36:37.126225 22666830645056 deprecation.py:506] From /linkhome/rech/genvsq01/ulz67kb/.conda/envs/SMAT_ADE/lib/python3.7/site-packages/tensorflow/python/ops/init_ops.py:1251: calling VarianceScaling.__init__ (from tensorflow.python.ops.init_ops) with dtype is deprecated and will be removed in a future version.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


n_train is 245 ; L = 36 ; D = 6 ; n_class = 14


W0105 22:36:47.621098 22666830645056 training_utils.py:1101] Output dense_8 missing from loss dictionary. We assume this was done on purpose. The fit and evaluate APIs will not be expecting any data to be passed to dense_8.
W0105 22:36:47.656050 22666830645056 deprecation.py:323] From /linkhome/rech/genvsq01/ulz67kb/.conda/envs/SMAT_ADE/lib/python3.7/site-packages/tensorflow/python/ops/math_grad.py:1250: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


n_epochs=3000, batch_size=2459, n_sup=2459, n_sup=0, steps=3000


/linkhome/rech/genvsq01/ulz67kb/.conda/envs/SMAT_ADE/lib/python3.7/site-packages/tensorflow/python/ops/gradients_util.py:93: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


...

Training Time for sample_rate (1.0000002) train_rate (0.9000002)  is 221
n_train is 2459 ; L = 36 ; D = 6 ; n_class = 14


W0105 22:59:47.853975 22666830645056 training_utils.py:1101] Output dense_8 missing from loss dictionary. We assume this was done on purpose. The fit and evaluate APIs will not be expecting any data to be passed to dense_8.


n_epochs=3000, batch_size=2459, n_sup=2459, n_sup=0, steps=3000


/linkhome/rech/genvsq01/ulz67kb/.conda/envs/SMAT_ADE/lib/python3.7/site-packages/tensorflow/python/ops/gradients_util.py:93: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Training Time for sample_rate (1.0000002) train_rate (1.0000002)  is 236


In [6]:
# C) vary dimension_rate
sample_rate, train_rate = 1, 1
dimension_rate = 0.05
save_running_time(ds_name, dataset, rep_output, sample_rate, train_rate, dimension_rate)
for dimension_rate in np.linspace(0.1, 1, 10):
    save_running_time(ds_name, dataset, rep_output, sample_rate, train_rate, dimension_rate)

W0111 18:02:44.278718 23347293087552 deprecation_wrapper.py:119] From /gpfsdswork/projects/rech/pch/ulz67kb/SMATE_MTS/utils/basic_modules.py:60: The name tf.keras.layers.CuDNNGRU is deprecated. Please use tf.compat.v1.keras.layers.CuDNNGRU instead.

W0111 18:02:44.281931 23347293087552 deprecation.py:506] From /linkhome/rech/genvsq01/ulz67kb/.conda/envs/SMAT_ADE/lib/python3.7/site-packages/tensorflow/python/ops/init_ops.py:1251: calling VarianceScaling.__init__ (from tensorflow.python.ops.init_ops) with dtype is deprecated and will be removed in a future version.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


x_train.shape is  (267, 144, 963)


W0111 18:02:50.455989 23347293087552 training_utils.py:1101] Output dense_8 missing from loss dictionary. We assume this was done on purpose. The fit and evaluate APIs will not be expecting any data to be passed to dense_8.
W0111 18:02:50.488826 23347293087552 deprecation.py:323] From /linkhome/rech/genvsq01/ulz67kb/.conda/envs/SMAT_ADE/lib/python3.7/site-packages/tensorflow/python/ops/math_grad.py:1250: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


n_epochs=300, batch_size=534, n_sup=267, n_sup=267, steps=300


/linkhome/rech/genvsq01/ulz67kb/.conda/envs/SMAT_ADE/lib/python3.7/site-packages/tensorflow/python/ops/gradients_util.py:93: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


...

Training Time for sample_rate (1.0000002) train_rate (1.0000002) dimension_size (866.0000002) is 82
x_train.shape is  (267, 144, 963)


W0111 18:30:19.099166 23347293087552 training_utils.py:1101] Output dense_98 missing from loss dictionary. We assume this was done on purpose. The fit and evaluate APIs will not be expecting any data to be passed to dense_98.


n_epochs=300, batch_size=534, n_sup=267, n_sup=267, steps=300


/linkhome/rech/genvsq01/ulz67kb/.conda/envs/SMAT_ADE/lib/python3.7/site-packages/tensorflow/python/ops/gradients_util.py:93: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Training Time for sample_rate (1.0000002) train_rate (1.0000002) dimension_size (963.0000002) is 88


In [13]:
x_train.shape

NameError: name 'x_train' is not defined